In [1]:
import pandas as pd
from pathlib import Path

# ===== 路径 =====
base_dir = Path(r"E:\1 ydd's space\ds\healthcare_job_analyze\healthcare_job_analyze\healthcare_job_dbt\data\split_cleaned_csv")

input_file = base_dir / "dim_company.csv"

out_company = base_dir / "dim_company_new.csv"
out_city = base_dir / "dim_city.csv"
out_country = base_dir / "dim_country.csv"

# ===== 读取 =====
df = pd.read_csv(input_file)

# ===== 1. country 维度 =====
dim_country = (
    df[["country"]]
    .drop_duplicates()
    .reset_index(drop=True)
)

dim_country["country_id"] = dim_country.index + 1

# ===== 2. city 维度 =====
dim_city = (
    df[["city", "country"]]
    .drop_duplicates()
    .merge(dim_country, on="country", how="left")
    .reset_index(drop=True)
)

dim_city["city_id"] = dim_city.index + 1

# ===== 3. company 表 =====
dim_company_new = (
    df
    .merge(
        dim_city[["city", "country", "city_id"]],
        on=["city", "country"],
        how="left"
    )
    .loc[:, ["company_id", "company_name", "city_id"]]
)

# ===== 保存 =====
dim_country[["country_id", "country"]].to_csv(out_country, index=False)
dim_city[["city_id", "city", "country_id"]].to_csv(out_city, index=False)
dim_company_new.to_csv(out_company, index=False)

print("维度表拆分完成")


维度表拆分完成
